In [3]:
!pip install -qqq accelerate transformers huggingface_hub langchain==0.0.166 pygpt4all==1.1.0 chromadb==0.3.22 llama-cpp-python==0.1.48 urllib3==1.26.6 pdfminer.six==20221105 > /dev/null

In [ ]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

In [ ]:
!wget https://huggingface.co/Pi3141/alpaca-native-7B-ggml/resolve/397e872bf4c83f4c642317a5bf65ce84a105786e/ggml-model-q4_0.bin

In [4]:
from langchain.chains import RetrievalQA
from langchain.embeddings import LlamaCppEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All, LlamaCpp
import os

In [5]:
 callbacks = [StreamingStdOutCallbackHandler()]

In [6]:
 llm = GPT4All(model="/content/ggml-gpt4all-j-v1.3-groovy.bin", 
               n_ctx=512, 
               backend='gptj', 
               callbacks=callbacks, 
               verbose=False)

In [11]:
llm("Who landed on the moon on July 21 1969")

?
The United States of America.

'?\nThe United States of America.'

In [15]:
from langchain import PromptTemplate,  LLMChain

template = """Use below text and answer the Question: {question}
{text}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, 
                        input_variables=["question","text"])

In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What is electroencephalography?"
text = """Electroencephalography (EEG) is a method to record an electrogram of the spontaneous electrical activity of the brain. The biosignals detected by EEG have been shown to represent the postsynaptic potentials of pyramidal neurons in the neocortex and allocortex.[1] It is typically non-invasive, with the EEG electrodes placed along the scalp (commonly called "scalp EEG") using the International 10-20 system, or variations of it. Electrocorticography, involving surgical placement of 
electrodes, is sometimes called "intracranial EEG". Clinical interpretation of
 EEG recordings is most often performed by visual inspection of the
  tracing or quantitative EEG analysis."""
print(llm_chain.run(question=question,text=text))

In [9]:
from chromadb.config import Settings

CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory="chromadb",
        anonymized_telemetry=False
)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("nomic-ai/gpt4all-j")

In [10]:
from langchain.embeddings import LlamaCppEmbeddings
llama = LlamaCppEmbeddings(model_path="/content/ggml-model-q4_0.bin")


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [11]:
from langchain.schema import Document

docs = [
    Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose", metadata={"year": 1993, "rating": 7.7, "genre": "science fiction","source":"local"}),
    Document(page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...", metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2,"source":"local"}),
    Document(page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea", metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6,"source":"local"}),
    Document(page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them", metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3,"source":"local"}),
    Document(page_content="Toys come alive and have a blast doing so", metadata={"year": 1995, "genre": "animated","source":"local"}),
    Document(page_content="Three men walk into the Zone, three men walk out of the Zone", metadata={"year": 1979, "rating": 9.9, "director": "Andrei Tarkovsky", "genre": "science fiction", "rating": 9.9,"source":"local"})
]

In [12]:
vectorstore = Chroma.from_documents(
    docs, llama
)

In [13]:
vectorstore.similarity_search("Movie on dinosaur")

[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3, 'source': 'local'}),
 Document(page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...', metadata={'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.2, 'source': 'local'}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'source': 'local'}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated', 'source': 'local'})]

In [14]:
from langchain.chains import RetrievalQAWithSourcesChain

In [51]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm, 
                                                    chain_type="stuff", 
                                                    retriever=vectorstore.as_retriever())

In [ ]:
chain({"question": "What movie did dinosaur escape"}, 
      return_only_outputs=True)

In [ ]:
from langchain.agents import load_huggingface_tool

tool = load_huggingface_tool("lysandre/hf-model-downloads")

print(f"{tool.name}: {tool.description}")

In [ ]:
tool.run("text-generation")

'gpt2'

In [ ]:
tool_tg = load_huggingface_tool("huggingface-tools/text-download")

In [ ]:
tool_tg("https://python.langchain.com/en/latest/modules/models/llms/integrations/huggingface_pipelines.html")

In [ ]:
tool_classify = load_huggingface_tool("Sj8287/Sentiment_Classification")